In [2]:
# 必要なライブラリをインポート
!pip install selenium

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime


def extract_stock_data(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    tooltip = soup.select_one("div.highcharts-tooltip table")
    if not tooltip:
        return None

    tds = tooltip.select("td")
    if len(tds) < 5:
        return None

    # 日付の整形
    try:
        y, m, d = map(int, tds[0].text.strip().split('/'))
        date = f"{y:04d}-{m:02d}-{d:02d}"
    except:
        return None

    # 株価4項目
    prices = [td.text.strip().replace(',', '') for td in tds[1:5]]
    return [date] + prices

def get_stock_values(driver, url):
    driver.get(url)
    time.sleep(5)  # ページ完全読み込み待ち（調整可）

    # グラフエリア取得
    chart = driver.find_element(By.CSS_SELECTOR, '.highcharts-grid')
    actions = ActionChains(driver)

    # グラフ中央にマウスを移動
    chart_size = chart.size
    width = chart_size['width']
    height = chart_size['height']
    center_x = width // 2
    center_y = height // 2
    start_x = center_x + width // 2

    actions.move_to_element_with_offset(chart, start_x, center_y).perform()
    time.sleep(0.5)

    data_points = []
    prev_date = ""

    for _ in range(chart_size['width']):
        actions.move_by_offset(-1, 0).perform()
        time.sleep(0.015)

        stock_data = extract_stock_data(driver)
        if stock_data and stock_data[0] != prev_date:
            data_points.append(stock_data)
            prev_date = stock_data[0]

    return data_points

def main():
    start_time = time.time()

    # ヘッドレスモードで起動するためのオプションを設定
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080") #ブラウザウィンドウサイズを広げる

    # Chromeを立ち上げる
    chrome_driver = webdriver.Chrome(options=chrome_options)

    try:
        url = 'https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month'
        stock_data_list = get_stock_values(chrome_driver, url)

        end_time = time.time()
        duration = end_time - start_time
        print(f"スクレイピング時間: {duration:.2f}秒\n")

        for data in stock_data_list:
            print(", ".join(data))
    finally:
        chrome_driver.quit()

if __name__ == "__main__":
    main()


スクレイピング時間: 231.62秒

2025-06-27, 始値: 39866.36, 高値: 40267.84, 安値: 39847.31, 終値: 40150.79
2025-06-26, 始値: 39072.14, 高値: 39615.59, 安値: 39056.14, 終値: 39584.58
2025-06-25, 始値: 38904.18, 高値: 38944.77, 安値: 38711.11, 終値: 38942.07
2025-06-24, 始値: 38779.18, 高値: 38990.11, 安値: 38665.61, 終値: 38790.56
2025-06-23, 始値: 38260.69, 高値: 38368.22, 安値: 38026.32, 終値: 38354.09
2025-06-20, 始値: 38472.25, 高値: 38646.16, 安値: 38362.54, 終値: 38403.23
2025-06-19, 始値: 38858.52, 高値: 38870.55, 安値: 38488.34, 終値: 38488.34
2025-06-18, 始値: 38364.16, 高値: 38885.15, 安値: 38364.16, 終値: 38885.15
2025-06-17, 始値: 38366.68, 高値: 38581.25, 安値: 38333.35, 終値: 38536.74
2025-06-16, 始値: 38056.86, 高値: 38342.52, 安値: 38055.52, 終値: 38311.33
2025-06-13, 始値: 38130.25, 高値: 38141.59, 安値: 37540.2, 終値: 37834.25
2025-06-12, 始値: 38324.93, 高値: 38407.57, 安値: 38102.05, 終値: 38173.09
2025-06-11, 始値: 38431.1, 高値: 38529.64, 安値: 38288.28, 終値: 38421.19
2025-06-10, 始値: 38278.17, 高値: 38495.7, 安値: 38139.1, 終値: 38211.51
2025-06-09, 始値: 38028.71, 高値: 38178.01, 安値: 38